In [16]:
%load_ext autoreload
%autoreload 2

import numpy as np
import molpy as mp
import molvis as mv
from pathlib import Path

canvas = mv.Molvis(
    width=600,
    height=600,
    background_color=(1.0, 1.0, 1.0),
)
data_path = Path("data/case1")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
struct_h = mp.Atomistic.from_frame(mp.io.read_pdb(data_path / "H.pdb"), name="H")
struct_n = mp.Atomistic.from_frame(mp.io.read_pdb(data_path / "N.pdb"), name="N")
struct_m = mp.Atomistic.from_frame(mp.io.read_pdb(data_path / "M.pdb"), name="M")
struct_p = mp.Atomistic.from_frame(mp.io.read_pdb(data_path / "P.pdb"), name="P")
struct_t = mp.Atomistic.from_frame(mp.io.read_pdb(data_path / "T.pdb"), name="T")
struct_tfsi = mp.Atomistic.from_frame(
    mp.io.read_pdb(data_path / "TFSI.pdb"), name="TFSI"
)

struct_n["net_charge"] = 1
struct_m["net_charge"] = -1
struct_tfsi["net_charge"] = -1

18
32
33
66
22
15


In [18]:
canvas.draw_struct(struct_h)

Molvis(background_color=[1.0, 1.0, 1.0], width=600)

In [19]:
monomer_h = mp.Monomer(
    struct_h, [mp.AnchorRule(name="tail", anchor=14, deletes=[15])]
)
monomer_n = mp.Monomer(
    struct_n,
    [
        mp.AnchorRule(name="head", anchor=3, deletes=[12]),
        mp.AnchorRule(name="tail", anchor=5, deletes=[11]),
    ],
)
monomer_m = mp.Monomer(
    struct_m,
    [
        mp.AnchorRule(name="head", anchor=20, deletes=[16]),
        mp.AnchorRule(name="tail", anchor=14, deletes=[9]),
    ],
)
monomer_p = mp.Monomer(
    struct_p,
    [
        mp.AnchorRule(name="head", anchor=0, deletes=[4]),
        mp.AnchorRule(name="tail", anchor=5, deletes=[7]),
    ],
)
monomer_t = mp.Monomer(
    struct_t, [mp.AnchorRule(name="head", anchor=1, deletes=[2])]
)

In [20]:
poly_builer = mp.builder.AmberToolsBuilder(workdir=data_path)
struct_poly = poly_builer.build_polymer(
    "blockN4M4P4",
    [monomer_h, monomer_n, monomer_m, monomer_p, monomer_t],
    list("HNNNNMMMMPPPPT"),
)

-I: Adding /opt/conda/envs/AmberTools25/dat/leap/prep to search path.
-I: Adding /opt/conda/envs/AmberTools25/dat/leap/lib to search path.
-I: Adding /opt/conda/envs/AmberTools25/dat/leap/parm to search path.
-I: Adding /opt/conda/envs/AmberTools25/dat/leap/cmd to search path.
-f: Source tleap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./tleap.in
----- Source: /opt/conda/envs/AmberTools25/dat/leap/cmd/leaprc.gaff
----- Source of /opt/conda/envs/AmberTools25/dat/leap/cmd/leaprc.gaff done
Log file: ./leap.log
Loading parameters: /opt/conda/envs/AmberTools25/dat/leap/parm/gaff.dat
Reading title:
AMBER General Force Field for organic molecules (Version 1.81, May 2017)
Loading Prep file: /workspaces/molpy/examples/data/case1/M/M.prepi
Loading parameters: /workspaces/molpy/examples/data/case1/M/M.frcmod
Reading force field modification type file (frcmod)
Reading title:
Remark line goes here
Loading Prep file: /workspaces/molpy/examples/data/case1/P/P.prepi
Loading parameters:

In [21]:
salt_builder = mp.builder.AmberToolsBuilder(workdir=data_path)
struct_litfsi = salt_builder.build_salt(
    "LiTFSI",
    mp.Monomer(struct_tfsi),
    "LI",
)

-I: Adding /opt/conda/envs/AmberTools25/dat/leap/prep to search path.
-I: Adding /opt/conda/envs/AmberTools25/dat/leap/lib to search path.
-I: Adding /opt/conda/envs/AmberTools25/dat/leap/parm to search path.
-I: Adding /opt/conda/envs/AmberTools25/dat/leap/cmd to search path.
-f: Source tleap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./tleap.in
----- Source: /opt/conda/envs/AmberTools25/dat/leap/cmd/leaprc.gaff
----- Source of /opt/conda/envs/AmberTools25/dat/leap/cmd/leaprc.gaff done
Log file: ./leap.log
Loading parameters: /opt/conda/envs/AmberTools25/dat/leap/parm/gaff.dat
Reading title:
AMBER General Force Field for organic molecules (Version 1.81, May 2017)
----- Source: /opt/conda/envs/AmberTools25/dat/leap/cmd/leaprc.water.tip3p
----- Source of /opt/conda/envs/AmberTools25/dat/leap/cmd/leaprc.water.tip3p done
Loading library: /opt/conda/envs/AmberTools25/dat/leap/lib/atomic_ions.lib
Loading library: /opt/conda/envs/AmberTools25/dat/leap/lib/solvents.lib
Loading

In [22]:
print(struct_poly)
print(struct_litfsi)

<Atomistic: 538 atoms>
<Atomistic: 16 atoms>


In [23]:
# === Load force field parameters ===
print("Loading AMBER force field parameters...")

# Read polymer force field
polymer_prmtop_path = data_path / "blockN4M4P4" / "blockN4M4P4.prmtop"
polymer_inpcrd_path = data_path / "blockN4M4P4" / "blockN4M4P4.inpcrd"
polymer_frame_with_ff = mp.io.read_amber(polymer_prmtop_path, polymer_inpcrd_path)
print(f"Loaded polymer force field from: {polymer_prmtop_path}")

# Read LiTFSI force field  
litfsi_prmtop_path = data_path / "LiTFSI" / "LiTFSI.prmtop"
litfsi_inpcrd_path = data_path / "LiTFSI" / "LiTFSI.inpcrd"
litfsi_frame_with_ff = mp.io.read_amber(litfsi_prmtop_path, litfsi_inpcrd_path)
print(f"Loaded LiTFSI force field from: {litfsi_prmtop_path}")

ff = mp.ForceField.from_forcefields("packed", polymer_frame_with_ff[1], litfsi_frame_with_ff[1])

Loading AMBER force field parameters...
Loaded polymer force field from: data/case1/blockN4M4P4/blockN4M4P4.prmtop
Loaded LiTFSI force field from: data/case1/LiTFSI/LiTFSI.prmtop


In [ ]:
import molpy.pack as mpk

# Define simulation box parameters
box_size = np.array([100.0, 100.0, 100.0])  # 40 x 40 x 40 Angstrom box
box_origin = np.array([0.0, 0.0, 0.0])

# Create packing session
packing_session = mpk.Session(workdir=data_path/"packed", packer="packmol")

# Add polymer molecules to the system
n_polymer_chains = 1  # Number of polymer chains
polymer_constraint = mpk.InsideBoxConstraint(box_size, box_origin)
packing_session.add_target(
    frame=struct_poly.to_frame(),
    number=n_polymer_chains,
    constraint=polymer_constraint
)

# Add LiTFSI salt molecules
n_litfsi = 10  # Number of LiTFSI molecules
# Create constraint that keeps salt molecules inside box but with minimum distance from polymers
salt_constraint = (
    mpk.InsideBoxConstraint(box_size, box_origin)
)
packing_session.add_target(
    frame=struct_litfsi.to_frame(),
    number=n_litfsi,
    constraint=salt_constraint
)

# Perform packing optimization
print("Starting molecular packing optimization...")
packed = packing_session.optimize(max_steps=2000, seed=42)
packed.box = mp.Box.cubic(box_size[0])

print(f"Packing completed!")
print(f"Final system contains:")
print(f"- {n_polymer_chains} polymer chains")
print(f"- {n_litfsi} LiTFSI molecules")
print(f"- Total atoms: {len(packed['atoms']['id']) if 'id' in packed['atoms'] else 'unknown'}")

Starting molecular packing optimization...
Cleaned up temporary file: data/case1/packed/run_local.sh
Packing completed!
Final system contains:
- 1 polymer chains
- 10 LiTFSI molecules
- Total atoms: 698


In [ ]:
mp.io.write_lammps(
    data_path/"packed",
    (packed.to_frame(), ff)
)